<a href="https://colab.research.google.com/github/Neiouo/Eric/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80_%E4%BD%9C%E6%A5%AD%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ks57O79IHqgCVQZ2djk7Fy2kY8r7TOY-0EY-3CNgrUM/edit?gid=1874159743#gid=1874159743')

from gspread_dataframe import set_with_dataframe
worksheet = gsheets.get_worksheet(0)
data = worksheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)

In [ ]:
# 定義一個分析函數
def analyze_data(df):
    # 範例提示詞: 根據數據的基本統計進行分析
    if df.empty:
        return "資料集是空的，無法進行分析。"

    # 顯示數據的基本資訊
    info = df.info()

    # 計算數據的基本統計量
    statistics = df.describe()

    # 檢查是否有缺失值
    missing_values = df.isnull().sum()

    # 確認是否有某個特定的列
    if 'target_column' in df.columns:
        target_column = df['target_column'].mean()  # 假設分析目標列的平均值
    else:
        target_column = None

    # 遍歷數據框架的每一列並檢查數據類型
    for column in df.columns:
        column_type = df[column].dtype
        print(f"列: {column} 類型: {column_type}")

    prompt = f"""
    請分析以下數據並以繁體中文回答:
    1. 數據框架的基本統計量：\n{statistics}
    2. 每列的缺失值數量：\n{missing_values}
    3. 是否有 'target_column' 列？其平均值為：{target_column}
    4. 數據框架的基本結構：\n{info}
    """
    return prompt

analysis_prompt = analyze_data(df)
print(analysis_prompt)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   遊戲名稱     100 non-null    object
 1   原價(美元)   100 non-null    object
 2   折扣率(%)   100 non-null    object
 3   折後價(美元)  100 non-null    object
 4   好評率(%)   100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB
列: 遊戲名稱 類型: object
列: 原價(美元) 類型: object
列: 折扣率(%) 類型: object
列: 折後價(美元) 類型: object
列: 好評率(%) 類型: object

    請分析以下數據並以繁體中文回答:
    1. 數據框架的基本統計量：
0      遊戲名稱 原價(美元) 折扣率(%) 折後價(美元) 好評率(%)
count   100    100    100     100    100
unique  100     99     10      98     91
top     遊戲1  58.76     70   14.11   68.9
freq      1      2     16       2      2
    2. 每列的缺失值數量：
0
遊戲名稱       0
原價(美元)     0
折扣率(%)     0
折後價(美元)    0
好評率(%)     0
dtype: int64
    3. 是否有 'target_column' 列？其平均值為：None
    4. 數據框架的基本結構：
None
    


In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('Gemini')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
response = model.generate_content(analysis_prompt)
respond = response.text.strip()

In [ ]:
from io import StringIO

# 轉換為 DataFrame
df_all = pd.read_csv(StringIO(response.text))

In [ ]:
print(df_all)

   這份數據分析報告顯示一個包含100筆遊戲資料的數據框架，其欄位包含遊戲名稱、原價(美元)、折扣率(%)、折後價(美元)和好評率(%)。
0                                         **數據特徵分析：**                 
1                  * **遊戲名稱:**  共有100種不同的遊戲，每個遊戲名稱唯一。                 
2   * **原價(美元):**  共有99個不同的價格，其中58.76美元出現兩次，是出現頻率最...                 
3   * **折扣率(%):**  只有10個不同的折扣率，其中70%折扣出現了16次，為最常見的...                 
4   * **折後價(美元):** 共有98個不同的折後價，其中14.11美元出現兩次，為出現頻率...                 
5   * **好評率(%):** 共有91個不同的好評率，其中68.9%出現兩次，為出現頻率最高的評價。                 
6                                         **數據品質分析：**                 
7                     * **缺失值:**  所有欄位都沒有缺失值，數據完整性良好。                 
8   * **目標變量('target_column'):**  數據中不存在目標變量('targ...                 
9                                           **整體結論：**                 
10  該數據集描述了100款遊戲的價格和好評率信息，折扣率相對集中，暗示著可能存在一些設定好的折扣...                 


In [ ]:
# 選擇要更新的工作表（選擇第一個工作表）
worksheet_allcommand = gsheets.get_worksheet(1)
# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet_allcommand, df_all)